# This is a notebook for querying the merged Eutro WB for Chl, Oxygen and Nutrients instance for the 1.0.0 Beacon release.
* You can run each cell individually by pressing "shift + enter".
* For more information, questions, bugs, please contact us on Slack:
    + https://join.slack.com/t/beacontechnic-wwa5548/shared_invite/zt-2dp1vv56r-tj_KFac0sAKNuAgUKPPDRg.

### 1. DD&AS token
In order to get access to the Beacon endpoint, you need to fill in your unique personal token between the " " in the cell below.

The token can be requested by writting an email to paul@maris.nl and robin@maris.nl. Then the token will be available in the DD&AS https://data.blue-cloud.org/search

![DDAS](images/ddas.png "DD&AS")
![token](images/beacon_token.png "token")

In [ ]:
Token = ""

### 2. Install and import the necessary packages
Install the following packages, if you have not already installed them in your environment:
* pip install requests
* pip install xarray
* pip install ipywidgets
* pip install cartopy
* pip install h5netcdf
* pip install netcdf4
* pip install scipy
* pip install packaging
* pip install plotly
* pip install numpy

Import the required packages

In [23]:
import requests
import json
import xarray as xr
import datetime
import pandas as pd
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np

### 3. Quering BEACON

#### 3.1 Retrieve the available columns from the Merged Eutrophication EOV's endpoint

Swagger page here: https://beacon-wb2-eutrophication.maris.nl/swagger/ 

In [24]:
responseinfo = requests.get("https://beacon-wb2-eutrophication.maris.nl/api/query/available-columns",
                            headers={"Authorization": f"Bearer {Token}"})


params = responseinfo.json()

#### 3.2 Available parameters
Below you can search through the available columns by entering text between the brackets of search_columns(" "). For the merged instance we need to look for the parameters under "COMMON"

In [25]:
def search_columns(search_term):
    search_term = search_term.lower()
    matches = [col for col in params if search_term in col.lower()]

    if matches:
        print("Matching columns:")
        for match in matches:
            print(match)
    else:
        print("No matching columns found.")


search_columns("")  # Enter your search term here

Matching columns:
CPHL
CPHL_QC
CPHL.standard_name
CPHL.units
CHLT
CHLT_QC
CHLT.standard_name
CHLT.units
DOXY
DOXY_QC
DOXY.standard_name
DOXY.units
DOX1
DOX1_QC
DOX1.standard_name
DOX1.units
DOX2
DOX2_QC
DOX2.standard_name
DOX2.units
OSAT
OSAT_QC
OSAT.standard_name
OSAT.units
NTRA
NTRA_QC
NTRA.standard_name
NTRA.units
NTAW
NTAW_QC
NTAW.standard_name
NTAW.units
PHOS
PHOS_QC
PHOS.standard_name
PHOS.units
PHOW
PHOW_QC
PHOW.standard_name
PHOW.units
SLCA
SLCA_QC
SLCA.standard_name
SLCA.units
SLCW
SLCW_QC
SLCW.standard_name
SLCW.units
PSAL
PSAL_QC
PSAL.standard_name
PSAL.units
TEMP
TEMP_QC
TEMP.standard_name
TEMP.units
LONGITUDE
LONGITUDE.standard_name
LONGITUDE.units
LATITUDE.standard_name
LATITUDE.units
LATITUDE
POSITION_QC
TIME
TIME_QC
DEPH
DEPH.standard_name
DEPH.units
DEPH_QC
PRES
PRES.standard_name
PRES.units
PRES_QC
dataset_id
SOURCE_BDI
SOURCE_BDI_DATASET_ID
COMMON_ORIGIN_CHLOROPHYLL
COMMON_ORIGIN_CHLOROPHYLL_QC
COMMON_ORIGIN_CHLOROPHYLL_STANDARD_NAME
COMMON_ORIGIN_CHLOROPHYLL_P01
COM

#### 3.3 Define your input parameters 
This test uses using the Marine Regions delimitation for the North East Atlantic area (General Sea Area): https://www.marineregions.org/gazetteer.php?p=details&id=5664

#### 3.4 Query body
Here you will create the query body based on your input parameters, you can add other "query_parameters" and "filters" to suit your needs. 
* For more query examples and explanations, you can take a look at https://github.com/maris-development/beacon-blue-cloud/.  

In [26]:
query_body = {
    "query_parameters": [
        {
            "column_name": "COMMON_OXYGEN_PER_VOLUME",
        },
        {
            "column_name": "COMMON_OXYGEN_PER_VOLUME_UNITS",
        },
        {
            "column_name": "COMMON_OXYGEN_PER_VOLUME_QC",
        },
        {
            "column_name": "COMMON_OXYGEN_PER_VOLUME_P01",
        },
        {
            "column_name": "COMMON_OXYGEN_PER_VOLUME_P06",
        },
        {
            "column_name": "COMMON_TIME",
            "alias": "datetime"
        },
        {
            "column_name": "COMMON_DEPTH",
            "alias": "DEPTH"
        },
        {
            "column_name": "COMMON_DEPTH_QC",
            "alias": "DEPTH.QC",
        },
        {
            "column_name": "COMMON_DEPTH_P01",
            "alias": "COMMON_DEPTH_P01",
        },
        {

            "column_name": "COMMON_DEPTH_P06",
            "alias": "COMMON_DEPTH_P06",
        },
        {
            "column_name": "COMMON_DEPTH_UNITS",
            "alias": "DEPTH.UNITS",
        },
        {
            "column_name": "COMMON_LONGITUDE",
            "alias": "LONGITUDE"
        },
        {
            "column_name": "COMMON_LATITUDE",
            "alias": "LATITUDE"
        },
        {
            "column_name": "COMMON_CHLOROPHYLL_PER_VOLUME",
        },
        {
            "column_name": "COMMON_CHLOROPHYLL_PER_VOLUME_P01",
        },
        {
            "column_name": "COMMON_CHLOROPHYLL_PER_VOLUME_P06",
        },
        {
            "column_name": "COMMON_CHLOROPHYLL_PER_VOLUME_QC",
        },
        {
            "column_name": "COMMON_CHLOROPHYLL_PER_VOLUME_STANDARD_NAME",
        },
        {
            "column_name": "COMMON_CHLOROPHYLL_PER_VOLUME_UNITS",
        },
        {
            "column_name": "SOURCE_BDI",
        },
        {
            "column_name": "SOURCE_BDI_DATASET_ID",
        },
        {
            "column_name": "COMMON_ODV_TAG",
        },
    ],
    "filters": [
        {
            "for_query_parameter": "datetime",
            "min": "2001-01-01T00:00:00",
            "max": "2001-12-31T00:00:00",
        },
        {
            "for_query_parameter": "DEPTH",
            "min": 0,
            "max": 1000
        },
        {
            "for_query_parameter": "LONGITUDE",
            "min": -44,
            "max": 5
        },
        {
            "for_query_parameter": "LATITUDE",
            "min": 15,
            "max": 60
        }
    ],
    "output": {
        "format": {
            "odv": {
                "longitude_column": {"column_name": "LONGITUDE"},
                "latitude_column": {"column_name": "LATITUDE"},
                "time_column": "datetime",
                "depth_column": {
                    "column_name": "DEPTH",
                    "comment": "",
                    "qf_column": "DEPTH.QC"
                },
                "data_columns": [
                    {
                        "column_name": "COMMON_OXYGEN_PER_VOLUME",
                        "comment": "",
                        "qf_column": "COMMON_OXYGEN_PER_VOLUME_QC"
                    },
                    {
                        "column_name": "COMMON_CHLOROPHYLL_PER_VOLUME",
                        "comment": "",
                        "qf_column": "COMMON_CHLOROPHYLL_PER_VOLUME_QC"
                    }
                ],
                "metadata_columns": [
                    {
                        "column_name": "COMMON_OXYGEN_PER_VOLUME_UNITS",
                    },
                    {
                        "column_name": "COMMON_CHLOROPHYLL_PER_VOLUME_UNITS",
                    },
                    {
                        "column_name": "DEPTH.UNITS",
                    },
                    {
                        "column_name": "COMMON_OXYGEN_PER_VOLUME_P01",
                    },
                    {
                        "column_name": "COMMON_CHLOROPHYLL_PER_VOLUME_P01",
                    },
                    {
                        "column_name": "COMMON_OXYGEN_PER_VOLUME_P06",
                    },
                    {
                        "column_name": "COMMON_CHLOROPHYLL_PER_VOLUME_P06",
                    },
                    {
                        "column_name": "SOURCE_BDI",
                    },
                    {
                        "column_name": "SOURCE_BDI_DATASET_ID",
                    }
                ],
                "qf_schema": "SEADATANET",
                "key_column": "COMMON_ODV_TAG",
                "archiving": "zip_deflate",
            }
        }
    }
}

#### 3.4 Retraiving the queried data
 This is the post request that is sent to Beacon with the above specified body.

In [27]:
response = requests.post("https://beacon-wb2-eutrophication.maris.nl/api/query", json.dumps(query_body), headers={
    "Authorization": f"Bearer {Token}",
    "Content-type": "application/json"
})

if response.status_code == 204:
    print("No data has been found for your query, please update your input fields above and run the notebook again.")
elif response.status_code != 200:
    print(response.text)

In [ ]:
# regionname = f"[{minlat},{minlon}],[{maxlat},{maxlon}]"

if not os.path.exists("./Beacon_V1.0.0_Output"):
    os.makedirs("Beacon_V1.0.0_Output")

open(
    f"./Beacon_V1.0.0_Output/EWB_merged_CHL_OXY_2001.zip",
    "wb",
).write(response.content)

14038931